## Statistical Methonds in Pandas

Statistical methods are used to perform basic statistical calculations on DataFrames.

In [ ]:
# import pandas

## Data loading 

In [ ]:
# load data and display head


### .Info

The .info() method in pandas is used to quickly get a summary of a DataFrame, including the data types of each column, the number of non-null values, and the memory usage. This method is particularly useful when you are exploring a dataset and want to get an overview of the structure of the DataFrame, such as how many missing values are in each column and what kind of data each column holds.

In [ ]:
# show info

### .describe

The .describe() method in pandas is used to generate descriptive statistics of a DataFrame or Series. It provides a summary of the central tendency, dispersion, and shape of the data distribution for each numeric column by default. This method is especially useful when you're exploring data and want to get a quick understanding of the summary statistics for your numerical columns, such as count, mean, standard deviation, min, max, and percentiles.

In [ ]:
# descriptive statistics

### Calculating Individual Column Statistics

In [ ]:
# individual column stats


In [ ]:
# column info


Common Descriptive statistics Methods


    .mode() -- the mode of the column
    .count() -- the count of the total number of entries in a column
    .std() -- the standard deviation for the column
    .var() -- the variance for the column
    .sum() -- the sum of all values in the column
    .cumsum() -- the cumulative sum, where each cell index contains the sum of all indices lower than, and including, itself.


In [ ]:
# Examples by class

### Summary Statistics for Categorical Columns

Obviously, we cannot calculate most summary statistics on columns that contain non-numeric data -- there's no way for us to find the mean of the letters in a column, for instance. However, there are some summary statistics we can use to help us better understand our categorical columns

### .unique() 

    shows us all the unique values contained in the column.

In [ ]:
# show all unique values in columns 


### .value_counts() 

    shows us a count for how many times each unique value is present in a dataset, giving us a feel for the distribution of values in the column.

In [ ]:
# show value counts 

## Calculating on the Fly with .apply() and .applymap()

Sometimes, we'll need to make changes to our dataset, or to compute functions on our data that aren't built-in to Pandas. We can do this by passing lambda values into the apply() method when working with Pandas series, and the .applymap() method when working with Pandas DataFrames.

Note that both of these do not mutate the original dataset -- instead, they return a copy of the Series or DataFrame containing the result.

In [ ]:
# convert data to strings with map / apply map


In [ ]:
# show info

### .apply()

In [ ]:
# view current prices


In [ ]:
# convert to kenya price 
current_rate =  130

In [ ]:
#  convert to kenyan price and assign to a new column


In [ ]:
# show top 5


## Data visualization with pandas 

In [ ]:
# plot saleprice 


In [ ]:
# another plot 
